In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/samplejobs/jobs.csv


In [2]:
# AUTHENTICATE WITH YOUR HF TOKEN
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
from huggingface_hub import login
login(token=hf_token)
print("HF AUTH COMPLETE — 70B GATE OPEN")

HF AUTH COMPLETE — 70B GATE OPEN


In [ ]:
#CMAKE_ARGS="-DGGML_CUDA=on" FORCE_CMAKE=1 pip install 'llama-cpp-python[server]'
#python3 -m llama_cpp.server --model models/7B/llama-model.gguf --n_gpu_layers 35

In [4]:
#!pip install -q bitsandbytes accelerate transformers sentencepiece protobuf
!pip install -q https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.16-cu124/llama_cpp_python-0.3.16-cp311-cp311-linux_x86_64.whl --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 551.3/551.3 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 96.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.3.5 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.3.5 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.

In [5]:
# Download model - only once
!wget -q https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf

In [8]:
# KAGGLE VICTORY — PRE-BUILT CU124 WHEEL (270 ROWS IN ~6–8 MIN)
# NO BUILD, NO DRAMA, FULL DUAL T4 OFFLOAD — TESTED DEC 2025

# Install from official releases (cu124 = Kaggle's CUDA version)
#!pip install -q https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.16-cu124/llama_cpp_python-0.3.16-cp311-cp311-linux_x86_64.whl --force-reinstall

from llama_cpp import Llama
import pandas as pd
import json, re
from tqdm import tqdm
tqdm.pandas()

# Download the perfect 8B GGUF (Q5_K_M — fits, fast, brutal)
# Comment out after model is downloaded
#!wget -q https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf

# FULL DUAL T4 OFFLOAD
llm = Llama(
    model_path="Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf",
    n_gpu_layers=-1,  # ALL to GPU
    n_batch=1024,
    n_ctx=4096,
    verbose=True  # see GPU offload logs
)

print("Llama-3.1-8B Q5_K_M CU124 FULLY LOADED — T4s ENGAGED")

# Upload your CSV
df = pd.read_csv('/kaggle/input/samplejobs/jobs.csv')  # fix path



ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    yes
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 2 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
  Device 1: Tesla T4, compute capability 7.5, VMM: yes
llama_model_load_from_file_impl: using device CUDA0 (Tesla T4) - 14992 MiB free
llama_model_load_from_file_impl: using device CUDA1 (Tesla T4) - 14992 MiB free
llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:               

Llama-3.1-8B Q5_K_M CU124 FULLY LOADED — T4s ENGAGED


CUDA : ARCHS = 500,520,530,600,610,620,700,720,750,800,860,870,890,900 | FORCE_MMQ = 1 | USE_GRAPHS = 1 | PEER_MAX_BATCH_SIZE = 128 | CPU : SSE3 = 1 | SSSE3 = 1 | AVX = 1 | AVX2 = 1 | F16C = 1 | FMA = 1 | BMI2 = 1 | LLAMAFILE = 1 | OPENMP = 1 | REPACK = 1 | 
Model metadata: {'quantize.imatrix.entries_count': '224', 'quantize.imatrix.dataset': '/training_dir/calibration_datav3.txt', 'quantize.imatrix.chunks_count': '125', 'quantize.imatrix.file': '/models_out/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct.imatrix', 'tokenizer.chat_template': '{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = "26 Jul 2024" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can sl

In [ ]:
# above script from 12-01-2025

In [9]:
# Neww updated temp and top_p make it a bit more creative
def roast(row):
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a battle-hardened senior engineer who has survived FAANG, startups, and government contracts. 
Your verdicts are sharp, varied, and brutally honest. 
Never start with "Another", "Yet another", "Classic", or "Typical". 
Use fresh venom every time. 
Return ONLY valid JSON. No explanations.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Title: {row.get('title','')}
Company: {row.get('company','')}
Location: {row.get('location','')}
Description: {str(row.get('description',''))[:2200]}

Return exactly:
{{
  "summary": "one savage sentence — be creative",
  "vibe": "faang_tier|hidden_gem|startup_chaos|corporate_zombie|avoid",
  "red_flags": ["max 4"],
  "green_flags": ["max 3"],
  "seniority": "junior|mid|senior|staff|principal",
  "match_score": 0-100
}}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""

    out = llm(prompt, max_tokens=350, temperature=0.95, top_p=0.92, stop=["<|eot_id|>"], echo=False)
    text = out['choices'][0]['text'].strip()

    # Aggressive JSON extraction
    start = text.find("{")
    end = text.rfind("}") + 1
    if start != -1 and end > start:
        try:
            return json.loads(text[start:end])
        except:
            pass
    
    # Fallback: look for any JSON-like blob
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(0))
        except:
            pass
    
    return {"summary": "parse failed", "vibe": "avoid"}

print("RE-ROASTING ALL 270 JOBS WITH BULLETPROOF PROMPT — ~6–8 MINUTES")
df['insights'] = df.progress_apply(roast, axis=1)

# Save the final victory
df.to_csv('SHOGUN9000K_FINAL_PERFECT_ROAST.csv', index=False)
print("VICTORY — DOWNLOAD SHOGUN9000K_FINAL_PERFECT_ROAST.csv — 99%+ BRUTAL JSON GUARANTEED")
# Below script is the OG, impressed with it.

RE-ROASTING ALL 270 JOBS WITH BULLETPROOF PROMPT — ~6–8 MINUTES


  0%|          | 0/270 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
llama_perf_context_print:        load time =     958.07 ms
llama_perf_context_print: prompt eval time =     957.31 ms /   473 tokens (    2.02 ms per token,   494.09 tokens per second)
llama_perf_context_print:        eval time =    2685.17 ms /   105 runs   (   25.57 ms per token,    39.10 tokens per second)
llama_perf_context_print:       total time =    3766.15 ms /   578 tokens
llama_perf_context_print:    graphs reused =        100
  1%|          | 2/270 [00:03<08:25,  1.89s/it]Llama.generate: 82 prefix-match hit, remaining 399 prompt tokens to eval
llama_perf_context_print:        load time =     958.07 ms
llama_perf_context_print: prompt eval time =     421.43 ms /   399 tokens (    1.06 ms per token,   946.78 tokens per s

VICTORY — DOWNLOAD SHOGUN9000K_FINAL_PERFECT_ROAST.csv — 99%+ BRUTAL JSON GUARANTEED


In [ ]:
# FIXED & FINAL — 99%+ PERFECT JSON — RUN THIS CELL ONLY
# Original - This one works great, just repetitious words being used
import json
import re
from tqdm import tqdm
tqdm.pandas()

def roast(row):
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a cynical senior engineer who has seen every tech scam. You return ONLY a single valid JSON object and nothing else — no explanations, no markdown, no extra text.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Title: {row.get('title','')}
Company: {row.get('company','')}
Location: {row.get('location','')}
Description: {str(row.get('description',''))[:2200]}

Return exactly this JSON structure and nothing else:
{{
  "summary": "one brutal sentence verdict",
  "vibe": "faang_tier|hidden_gem|startup_chaos|corporate_zombie|avoid",
  "red_flags": ["flag1", "flag2"],
  "green_flags": ["flag1"],
  "seniority": "junior|mid|senior|staff|principal",
  "match_score": 0-100
}}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""
    

    out = llm(prompt, max_tokens=350, temperature=0.82, stop=["<|eot_id|>"], echo=False)
    
    text = out['choices'][0]['text'].strip()

    # Aggressive JSON extraction
    start = text.find("{")
    end = text.rfind("}") + 1
    if start != -1 and end > start:
        try:
            return json.loads(text[start:end])
        except:
            pass
    
    # Fallback: look for any JSON-like blob
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(0))
        except:
            pass
    
    return {"summary": "parse failed", "vibe": "avoid"}

print("RE-ROASTING ALL 270 JOBS WITH BULLETPROOF PROMPT — ~6–8 MINUTES")
df['insights'] = df.progress_apply(roast, axis=1)

# Save the final victory
df.to_csv('SHOGUN9000K_FINAL_PERFECT_ROAST.csv', index=False)
print("VICTORY — DOWNLOAD SHOGUN9000K_FINAL_PERFECT_ROAST.csv — 99%+ BRUTAL JSON GUARANTEED")